In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning )

In [170]:
rel_ref = './kaggle/input/brazilian-ecommerce/' # then + file name
olist_customers_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')
olist_orders_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')
olist_order_items_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
olist_geolocation_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv')
olist_orders_dataset['order_purchase_timestamp'] = pd.to_datetime(olist_orders_dataset['order_purchase_timestamp'])
olist_orders_dataset['order_delivered_customer_date'] = pd.to_datetime(olist_orders_dataset['order_delivered_customer_date'])
olist_sellers_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv')
product_data = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_products_dataset.csv')
olist_customers_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')
olist_geolocation_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv')
olist_geolocation_dataset = olist_geolocation_dataset.groupby('geolocation_zip_code_prefix').agg({"geolocation_lat":"mean",
                                                                                                 'geolocation_lng':'mean',
                                                                                                  })

olist_order_payments_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv')
olist_geolocation_dataset = olist_geolocation_dataset.reset_index()
filter1 = olist_orders_dataset['order_purchase_timestamp'] > '2016-12-31'
filter2 = olist_orders_dataset['order_purchase_timestamp'] < '2018-09-1'
filter3 = ~olist_orders_dataset['order_status'].isin(['unavailable' ,'canceled'])  ## not in condition
olist_orders_dataset = olist_orders_dataset[(filter1 & filter2 & filter3)] ### order between 2017 and 2018 and ordered.

In [171]:
olist_orders_dataset.shape

(97910, 8)

In [172]:
olist_orders_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97910 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       97910 non-null  object        
 1   customer_id                    97910 non-null  object        
 2   order_status                   97910 non-null  object        
 3   order_purchase_timestamp       97910 non-null  datetime64[ns]
 4   order_approved_at              97891 non-null  object        
 5   order_delivered_carrier_date   97306 non-null  object        
 6   order_delivered_customer_date  96203 non-null  datetime64[ns]
 7   order_estimated_delivery_date  97910 non-null  object        
dtypes: datetime64[ns](2), object(6)
memory usage: 6.7+ MB


In [173]:
olist_orders_dataset.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  19
order_delivered_carrier_date      604
order_delivered_customer_date    1707
order_estimated_delivery_date       0
dtype: int64

In [174]:
olist_orders_dataset['order_purchase_timestamp'] = pd.to_datetime(olist_orders_dataset['order_purchase_timestamp'])
olist_orders_dataset['order_delivered_customer_date'] = pd.to_datetime(olist_orders_dataset['order_purchase_timestamp'])

In [175]:
olist_orders_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97910 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       97910 non-null  object        
 1   customer_id                    97910 non-null  object        
 2   order_status                   97910 non-null  object        
 3   order_purchase_timestamp       97910 non-null  datetime64[ns]
 4   order_approved_at              97891 non-null  object        
 5   order_delivered_carrier_date   97306 non-null  object        
 6   order_delivered_customer_date  97910 non-null  datetime64[ns]
 7   order_estimated_delivery_date  97910 non-null  object        
dtypes: datetime64[ns](2), object(6)
memory usage: 6.7+ MB


In [176]:
olist_customers_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [177]:
olist_orders_dataset

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-02 10:56:33,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-07-24 20:41:37,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-08 08:38:49,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-11-18 19:28:06,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-13 21:18:39,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-09 09:54:05,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-06 12:58:58,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-08-27 14:46:43,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-08 21:28:27,2018-02-15 00:00:00


In [178]:
order_counts = olist_orders_dataset['order_status'].value_counts().reset_index()
order_counts.to_csv("order_status_counts.csv", index=False)  # Save without the index columna

In [28]:
print('numbers of orders : ', olist_orders_dataset.shape[0])
print('first order : '  , olist_orders_dataset['order_purchase_timestamp'].min())
print('last order : ' , olist_orders_dataset['order_purchase_timestamp'].max())

numbers of orders :  99441
first order :  2016-09-04 21:15:19
last order :  2018-10-17 17:30:18


In [62]:
c = olist_order_payments_dataset[[ 'payment_sequential',
       'payment_installments', 'payment_value'] ].describe()

In [ ]:
product_data

In [46]:
olist_order_payments_dataset.describe()

,payment_sequential,payment_installments,payment_value
count,103886.000000,103886.000000,103886.000000
mean,1.092679,2.853349,154.100380
std,0.706584,2.687051,217.494064
min,1.000000,0.000000,0.000000
25%,1.000000,1.000000,56.790000
50%,1.000000,1.000000,100.000000
75%,1.000000,4.000000,171.837500
max,29.000000,24.000000,13664.080000


,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


In [49]:
product_data['volume'] = (
    product_data['product_width_cm']  *
    product_data['product_height_cm'] *
    product_data['product_length_cm'] 
)

In [63]:
b = product_data.describe()[['volume', 'product_weight_g' ] ]

In [58]:
olist_orders_review_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv')


In [64]:
a = olist_orders_review_dataset[['review_score']].describe()

In [66]:
result = pd.merge(a, b, left_index=True, right_index=True, how='inner')

In [68]:
result = pd.merge(result,c,left_index=True,right_index=True)

In [69]:
result

,review_score,volume,product_weight_g,payment_sequential,payment_installments,payment_value
count,99224.000000,32949.000000,32949.000000,103886.000000,103886.000000,103886.000000
mean,4.086421,16564.096695,2276.472488,1.092679,2.853349,154.100380
std,1.347579,27057.041650,4282.038731,0.706584,2.687051,217.494064
min,1.000000,168.000000,0.000000,1.000000,0.000000,0.000000
25%,4.000000,2880.000000,300.000000,1.000000,1.000000,56.790000
50%,5.000000,6840.000000,700.000000,1.000000,1.000000,100.000000
75%,5.000000,18480.000000,1900.000000,1.000000,4.000000,171.837500
max,5.000000,296208.000000,40425.000000,29.000000,24.000000,13664.080000


In [129]:
a = olist_customers_dataset.isna().sum().reset_index() 
a

,index,0
0,customer_id,0
1,customer_unique_id,0
2,customer_zip_code_prefix,0
3,customer_city,0
4,customer_state,0


In [130]:
b = olist_orders_dataset.isna().sum().reset_index()
b

,index,0
0,order_id,0
1,customer_id,0
2,order_status,0
3,order_purchase_timestamp,0
4,order_approved_at,160
5,order_delivered_carrier_date,1783
6,order_delivered_customer_date,2965
7,order_estimated_delivery_date,0


In [179]:
olist_order_items_dataset.dropna(inplace=True)


In [133]:
d = olist_geolocation_dataset.isna().sum().reset_index()
d

,index,0
0,geolocation_zip_code_prefix,0
1,geolocation_lat,0
2,geolocation_lng,0


In [135]:
e = product_data.isna().sum().reset_index()
e

,index,0
0,product_id,0
1,product_category_name,610
2,product_name_lenght,610
3,product_description_lenght,610
4,product_photos_qty,610
5,product_weight_g,2
6,product_length_cm,2
7,product_height_cm,2
8,product_width_cm,2
9,volume,2


In [137]:
f  = olist_order_payments_dataset.isna().sum().reset_index()
f

,index,0
0,order_id,0
1,payment_sequential,0
2,payment_type,0
3,payment_installments,0
4,payment_value,0


In [138]:
g = olist_sellers_dataset.isna().sum().reset_index()
g

,index,0
0,seller_id,0
1,seller_zip_code_prefix,0
2,seller_city,0
3,seller_state,0


In [139]:
h = Product_category_name_translation.isna().sum().reset_index()
h

,index,0
0,product_category_name,0
1,product_category_name_english,0


In [124]:
Product_category_name_translation = pd.read_csv('./kaggle/input/brazilian-ecommerce/product_category_name_translation.csv')

In [154]:
result = pd.concat([a,b,c,d,e,f,g,h], axis=0)
result = result.rename(columns={"index":"columns" , 0:"NA values"})
result[result['NA values'] > 0]

,columns,NA values
4,order_approved_at,160
5,order_delivered_carrier_date,1783
6,order_delivered_customer_date,2965
1,product_category_name,610
2,product_name_lenght,610
3,product_description_lenght,610
4,product_photos_qty,610
5,product_weight_g,2
6,product_length_cm,2
7,product_height_cm,2


In [ ]:
rel_ref = './kaggle/input/brazilian-ecommerce/' # then + file name
olist_customers_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')
olist_orders_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')
olist_order_items_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
olist_geolocation_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv')
olist_orders_dataset['order_purchase_timestamp'] = pd.to_datetime(olist_orders_dataset['order_purchase_timestamp'])
olist_orders_dataset['order_delivered_customer_date'] = pd.to_datetime(olist_orders_dataset['order_delivered_customer_date'])
olist_sellers_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv')
product_data = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_products_dataset.csv')
olist_customers_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')
olist_geolocation_dataset = pd.read_csv('./kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv')
olist_geolocation_dataset = olist_geolocation_dataset.groupby('geolocation_zip_code_prefix').agg({"geolocation_lat":"mean",
                                                                                                 'geolocation_lng':'mean',
                                                                                                  })



In [157]:
olist_sellers_dataset.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [160]:
len(olist_sellers_dataset['seller_state'].value_counts())

23

In [161]:
len(olist_sellers_dataset['seller_city'].value_counts())

611

In [164]:
len(olist_customers_dataset['customer_state'].value_counts())

27

In [165]:
len(olist_customers_dataset['customer_city'].value_counts())

4119

In [181]:
new_data  = pd.merge(olist_orders_dataset,olist_order_items_dataset , on='order_id',)

In [182]:
new_data

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-02 10:56:33,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-07-24 20:41:37,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-08 08:38:49,2018-09-04 00:00:00,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-11-18 19:28:06,2017-12-15 00:00:00,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-13 21:18:39,2018-02-26 00:00:00,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111747,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-06 12:58:58,2018-03-02 00:00:00,1,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,2018-02-12 13:10:37,174.90,20.10
111748,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-08-27 14:46:43,2017-09-27 00:00:00,1,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,2017-09-05 15:04:16,205.99,65.02
111749,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-08 21:28:27,2018-02-15 00:00:00,1,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59
111750,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-08 21:28:27,2018-02-15 00:00:00,2,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59
